In [1]:
def print_graph(graph):
    for k, v in graph.iteritems():        
        print k, ':', v

In [2]:
def duplicate_and_transform_graph(graph, T):
    l = len(graph)
    new_graph = {}
    offset = max(graph.keys()) + 1
    to_duplicant = lambda x: None if x is None else x + offset
    for k, (formula, (cyclic, neighbours)) in graph.iteritems():
        new_graph[k] = formula, (cyclic, neighbours)
        new_graph[to_duplicant(k)] = T + formula, (cyclic, [to_duplicant(x) for x in neighbours])  
    return new_graph

In [3]:
from copy import copy
def clone_graph(graph):
    return {k: (f, (m, copy(ns))) for k, (f, (m, ns)) in graph.iteritems()}

In [4]:
def apply_prefix_rules(graph, rules):
    new_graph = []
    for formula, (mod, neighbours) in graph:
        for (prefix, replacement) in rules:
            if formula.startswith(rule):
                new_formula = replacement + formula[len(prefix)+1:]
            else:
                new_formula = formula
            new_graph.append((new_formula, (mod, neighbours)))

In [5]:
def intersection_with_indices(l1, l2):
    common_members = set(l1).intersection(l2)
    return [(x, l1.index(x), l2.index(x)) for x in common_members]

In [6]:
def pairwise_induction(pairs):
    for (a, b) in pairs:
        if a is not None and b is not None:
            yield (a, b)
        elif a is None:
            yield (b, b)
        else:
            yield (a, a)
            

In [7]:
def pad(l, n, padding_direction=1, v=None):
    from itertools import repeat
    if len(l) > l:
        return l
    padding = list(repeat(v, n - len(l)))
    if padding_direction > 0:
        return l + padding
    else:
        return padding + l

In [8]:
def has_repeating_values(l):
    without_nones = filter(lambda x: x is not None, l)
    return len(l) > len(set(l))

In [9]:
def is_sorted(l):
    return all([x < y for x, y in zip(l, l[1:])])

In [10]:
def padding_zip(l1, l2, padding_direction=1):
    length = max(len(l1), len(l2))
    return zip(pad(l1, length, padding_direction),
               pad(l2, length, padding_direction))

In [11]:
def mirror(tpls):
    if not tpls:
        return None
    (c, l) = tpls
    return (c, [(b, a) for (a, b) in l])

In [12]:
def common_cycle3(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if not c1 or not c2:
        #print 'Non cyclic'
        return None
    if len(l1) != len(l2):
        #print 'Non matching lengths {} {}'.format(l1, l2)
        return None
    common_members = set(l1).intersection(l2)
    if len(common_members) == 0:
        print 'No common members: {}, {}'.format(l1, l2)
        return None
    m0 = common_members.pop()
    skew1 = l1.index(m0)
    skew2 = l2.index(m0)
    candidate = list(pairwise_induction([(l1[(i + skew1) % len(l1)], l2[(i + skew2) % len(l2)])
                                         for i in range(len(l1))]))
    if any([x != y and (x in common_members or y in common_members) for x, y in candidate]):
        #print [x != y and (x in common_members or y in common_members) for x, y in candidate]
        #print 'Non matching cycles {}'.format(candidate)
        return None
    return (True, candidate)

In [13]:
def common_cycle2(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if c1 and not c2:
        return common_cycle2(s2, s1)
        return mirror(common_cycle2(s2, s1))
    common_numbers = set(l1).intersection(l2)
    def to_letter(n, wild='#'):
        if n > 26:
            raise Exception('Illegal ordinal: {}'.format(n))
        return chr(ord('A') + n) if n in common_numbers else wild
    if c1 and c2:
        regex = ''.join([to_letter(n) for n in l1 ]) + r'(?={})'.format(to_letter(l1[0]))
        matched_string = ''.join([to_letter(n) for n in l2 + l2])
    elif c2:
        regex = ''.join([to_letter(n) for n in l1])
        matched_string = ''.join([to_letter(n) for n in l2 + l2])
    else:
        # not c1 and not c2        
        regex = r'-?'.join([to_letter(n).strip('#'). for n in l1])
        matched_string = '{x}-{x}'.format(x=''.join([to_letter(n) for n in l2]))
    match = re.search(regex, matched_string)
    if not match:
        return None
    
    return (regex, matched_string, match is not None and (match.group(0), match.start(), match.end()))

SyntaxError: invalid syntax (<ipython-input-13-34d38f8dbf58>, line 21)

In [14]:
def common_cycle(s1, s2):
    from itertools import repeat, chain
    (c1, l1) = s1
    (c2, l2) = s2
    if c1 and c2 and len(l1) != len(l2):
        print 'Cannot combine closed cycles of different lengths'
    if has_repeating_values(l1):
        raise Exception('Non unique members in cycle 1')
    if has_repeating_values(l2):
        raise Exception('Non unique members in cycle 2')
    tpls = sorted(intersection_with_indices(l1, l2), key= lambda x: x[1:2])
    if len(tpls) == 0:
        print 'Cannot combine non intersecting sections'
        return None
    if is_sorted([x[2] for x in tpls]):
        print tpls
        result = list(padding_zip(l1[0:tpls[0][1]], l2[0:tpls[0][2]], padding_direction=-1))
        for prv, tpl in zip(tpls, tpls[1:]):
            if tpl[1] - prv[1] == tpl[2] - prv[2]:
                result = result + list(zip(l1[prv[1]:tpl[1]], l2[prv[2]:tpl[2]]))
            else:
                print 'Misatch: {} - {} != {} - {}'.format(tpl[1], prv[1], tpl[2], prv[2])
                return None
        result = result + padding_zip(l1[tpls[-1][1]:], l2[tpls[-1][2]:])
        return (c1 or c2, list(pairwise_induction(result)))
    result = []
    overboard = False
    for i, prv, tpl in zip(range(len(tpls)), tpls, tpls[1:]):
        if tpl[2] < prv[2]:
            ''' List 2 performed a whole cycle '''
            if overboard:
                raise Exception('Ordering is not cyclic')
            overboard = True
            gap = (tpl[1] - prv[1]) - (tpl[2] + len(l2) - prv[2])
            if c2 and gap != 0:
                raise Exception('Mismatch in common index {} (when reaching end of l2, sorted)'.format(i))
            if gap < 0:
                raise Exception('Cannot negative pad in index {} (when reaching end of l2, sorted)'.format(i))
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], len(l2))] + \
                 list(repeat(None, gap)) + \
                 [l2[i] for i in range(0, tpl[2])]
            result = result + zip(s1, s2)
        elif tpl[2] - prv[2] != tpl[1] - prv[1]:
            raise Exception('Mismatch in index {} (sorted)'.format(i))
        else:
            s1 = [l1[i] for i in range(prv[1], tpl[1])]
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            result = result + zip(s1, s2)
    result_is_closed = c1 or c2 or overboard
    if result_is_closed:
        ''' List 1 must perform a whole cycle '''
        prv = tpls[-1]
        tpl = tpls[0]
        gap = (tpl[2] - prv[2]) - (tpl[1] + len(l1) - prv[1])
        if c1:
            if gap != 0:
                raise Exception('Mismatch in when reaching end of l1')
        if gap < 0:
            if tpl[2] < prv[2]:
                print 'Gap : {}'.format(gap)
                result.append((l1[-1], l2[-1]))
            else:
                raise Exception('Cannot negative pad when reaching end of l1')
        else:
            s2 = [l2[i] for i in range(prv[2], tpl[2])]
            s1 = [l1[i] for i in range(prv[1], len(l1))] + \
                 list(repeat(None, gap)) + \
                 [l1[i] for i in range(0, tpl[1])]
            result = result + zip(s1, s2)
    else:
        result = padding_zip(l1[0:tpls[0][1]], l2[0:tpls[0][2]], padding_direction=-1) + \
                 result + \
                 padding_zip(l1[tpls[-1][1]:], l2[tpls[-1][2]:])
    return (result_is_closed, list(pairwise_induction(result)))                

In [15]:
def drop(n, it):
    for i in range(n):
        it.next()
    while True:
        yield it.next()

In [16]:
def compare_cyclic(l1, l2):
    from itertools import cycle
    if set(l1) != set(l2) or len(l1) != len(l2):
        return False
    for a,b in zip(l1, drop(l2.index(l1[0]), cycle(l2))):
        if a != b:
            return False
    return True

In [17]:
def test(a, b, expected):
    cc = common_cycle(a, b)
    (c, result) = cc
    rev = common_cycle(b, a)
    (cm, mirrored_result) = rev
    (cme, mirrored_expected) = mirror(expected)
    if not compare_cyclic(result, expected[1]) or c != expected[0]:
        print '{} and \n{} resulted in \n{} and did not result in \n{}'.format(a, b, result, expected)
    if not compare_cyclic(mirrored_result, mirrored_expected) or cm != cme:
        print '{} and \n{} resulted in \n{} and did not result in \n{}'.format(b, a, mirrored_result, mirror(expected))        

In [18]:
test((False, [0,1]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2)]))
test((False, [0,1,2,3]), (False, [2,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,2]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [1,None,3]),(False, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [0,1,2,3]), (False, [2,None,0]),(True, [(0,0), (1,1), (2, 2), (3,3)]))
test((False, [1,2]), (False, [2,3]),(False, [(1,1), (2,2), (3,3)]))
test((False, [0,1]), (True, [0,1]),(True, [(0,0), (1,1)]))
test((True, [11, 2, 0]), (False, [2, 0, 5]), (True, [(11, 5), (2, 2), (0, 0)]))

[(1, 1, 0)]
[(1, 0, 1)]
[(2, 2, 0), (3, 3, 1)]
[(2, 0, 2), (3, 1, 3)]
[(1, 1, 0), (2, 2, 1)]
[(1, 0, 1), (2, 1, 2)]
[(1, 1, 0), (3, 3, 2)]
[(1, 0, 1), (3, 2, 3)]
[(2, 1, 0)]
[(2, 0, 1)]
[(0, 0, 0), (1, 1, 1)]
[(0, 0, 0), (1, 1, 1)]
[(2, 1, 0), (0, 2, 1)]
[(2, 0, 1), (0, 1, 2)]
(True, [11, 2, 0]) and 
(False, [2, 0, 5]) resulted in 
[(11, 11), (2, 2), (0, 0), (5, 5)] and did not result in 
(True, [(11, 5), (2, 2), (0, 0)])
(False, [2, 0, 5]) and 
(True, [11, 2, 0]) resulted in 
[(11, 11), (2, 2), (0, 0), (5, 5)] and did not result in 
(True, [(5, 11), (2, 2), (0, 0)])


In [19]:
def common_member(l1, l2):    
    for i1 in range(len(l1)):
        x1 = l1[i1]
        if x1 in l2:
            return (i1, l2.index(l1[i1]))
    return (-1, -1)

In [20]:
def index_or_identity(l, i):
    return l[i] if i in l else i

In [21]:
def without(l, indices):
    return [l[i] for i in range(len(l)) if i not in indices]

def without_keys(d, keys):
    return {k: v for k, v in d.iteritems() if k not in keys}

In [22]:
def replace(graph, merging_table):
    replacer = lambda x: merging_table[x] if x in merging_table else x
    return {
        k: (f, (c, map(replacer, n))) for k, (f, (c, n)) in graph.iteritems()
    }

def scan_for_errors(graph):
    for k, (f, (c, n)) in graph.iteritems():
        if len(n) != len(set(n)):
            print_graph(graph)
            raise Exception('Non unique members in node {}'.format(k))

In [23]:
from functools import partial
from copy import copy
def merge_vertices(graph, i1, i2, merging_table, countdown=100):
    if countdown < 0:
        print 'Stack overflow'
        import sys
        sys.exit(1)
    #print 'Merging {} ({}) into {} ({})'.format(i1, graph[i1][0], i2, graph[i2][0])
    #print 'with'
    #print 'Graph: '
    #print_graph(graph)
    #print 'Merging Table: '
    #print merging_table
    '''
    vertex of index i1 is merged into i2
    i1 - source index
    i2 - destination index
    '''
    if i1 == i2:
        #print 'Not merging'
        #print 'After merging {} into {}'.format(i1, i2)
        #print graph.keys()
        return graph
    f1, (c1, n1) = graph[i1]
    f2, (c2, n2) = graph[i2]
    mapper = partial(map, partial(index_or_identity, merging_table))
    cc = common_cycle3((c1, mapper(n1)), (c2, mapper(n2)))
    #print cc
    if cc is None:
        #print 'Not merging'
        #print 'After merging {} into {}'.format(i1, i2)
        #print graph.keys()
        return graph
    new_graph = clone_graph(graph)
    new_graph[i2] = (f2, (cc[0], [x[1] for x in cc[1]]))
    local_merging_table = {}
    for src, dst in cc[1]:
        if src != dst:
            local_merging_table[src] = dst
    combined_merging_table = copy(local_merging_table)
    combined_merging_table.update(merging_table)
    combined_merging_table[i1] = i2
    for src, dst in local_merging_table.iteritems():
        if dst in new_graph and src in new_graph:
            new_graph = merge_vertices(new_graph, src, dst, combined_merging_table, countdown=countdown-1)
    #print 'combined_merging_table: {}'.format(combined_merging_table)
    #print 'After merging {} into {}'.format(i1, i2)
    #print without_keys(replace(new_graph, combined_merging_table), {i1}).keys()
    return without_keys(replace(new_graph, combined_merging_table), {i1})

In [24]:
from collections import defaultdict

def progress_graph(initial_graph, path, rules):
    graph = initial_graph
    for c in path:
        print 'Applying {}'.format(c)
        grouped_by_formula = defaultdict(list)
        new_graph = duplicate_and_transform_graph(graph, c)
        #print 'Duplicated graph: '
        #print_graph(new_graph)
        for k in sorted(new_graph.keys()):
            (f, x)  = new_graph[k]
            cnt = True
            new_formula = f
            while cnt:
                brk = True
                for fr, to in rules:
                    if fr in new_formula:
                        new_formula = new_formula.replace(fr, to)
                        brk = False
                        #print 'Reducing {} : {} into {} ({} -> {})'.format(k, f, new_formula, fr, to)
                cnt = cnt and not brk
            grouped_by_formula[new_formula].append(k)
            #import pdb
            #pdb.set_trace()
            new_graph[k] = (new_formula, x)    
        #print 'Reduced graph: '
        #print_graph(new_graph)
        merging_table = {}
        for f, group in grouped_by_formula.iteritems():
            if len(group) == 2:
                a = group[1]
                b = group[0]
                while b in merging_table:
                    b = merging_table[b]
                merging_table[a] = b
            elif len(group) > 2:
                print f, group
                #print 'A group larger than 2'
                break
        #print 'Merging the following: {}'.format(merging_table) 
        for src, dst in merging_table.iteritems():
            if dst in new_graph:
                new_graph = merge_vertices(new_graph, src, dst, merging_table)
                #print new_graph.keys()
            else:
                print "Not merging {} -> {}".format(src, dst)
        graph = new_graph
        #print 'Result graph: '
        #print_graph(new_graph)
    return graph

In [25]:
progress_graph(initial_graph, path, rules)

NameError: name 'initial_graph' is not defined

In [26]:
actions = ['a', 'b']
bucky_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('ababbP', 'bP')]


bucky_initial_graph = {
    0: ('P', (True, [4, 1, 8])),
    1: ('bP', (True, [0, 2, 5])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 4, None])),
    4: ('bbbbP', (True, [3, 0, None])),
    5: ('aaP', (True, [1, None, 6])),
    6: ('aabP', (True, [5, None, 7])),
    7: ('aP', (True, [6, None, 8])),
    8: ('abP', (True, [7, None, 0])),    
}

bucky_path = 'bbbbabbbaaaa'
bucky = progress_graph(bucky_initial_graph, bucky_path, bucky_rules)

Applying b
Applying b
Applying b
Applying b
Applying a
Applying b
Applying b
Applying b
Applying a
Applying a
Applying a
Applying a


In [27]:
del path

NameError: name 'path' is not defined

In [28]:
bucky_initial_graph

{0: ('P', (True, [4, 1, 8])),
 1: ('bP', (True, [0, 2, 5])),
 2: ('bbP', (True, [1, 3, None])),
 3: ('bbbP', (True, [2, 4, None])),
 4: ('bbbbP', (True, [3, 0, None])),
 5: ('aaP', (True, [1, None, 6])),
 6: ('aabP', (True, [5, None, 7])),
 7: ('aP', (True, [6, None, 8])),
 8: ('abP', (True, [7, None, 0]))}

In [29]:
def find_cycle(g, r, c, d=0):
    if d < 0:
        return 0
    if r in g[c][1]:
        return d
    else:
        return 1 + find_cycle



In [30]:
actions = ['a', 'b']
dodca_rules = [('aaa', ''),
         ('bbbbb', ''),
         ('aP', 'P'),
         ('bbbbP', 'abP')]


dodca_initial_graph = {
    0: ('P', (True, [4, 1, None])),
    1: ('bP', (True, [0, 2, None])),
    2: ('bbP', (True, [1, 3, None])),
    3: ('bbbP', (True, [2, 4, None])),
    4: ('bbbbP', (True, [3, 0, None])),
}

path = 'aabbbbaabbbabababa'
progress_graph(dodca_initial_graph, path, dodca_rules)

Applying a
Applying a
Applying b
Applying b
Applying b
Applying b
Applying a
Applying a
Applying b
Applying b
Applying b
Applying a
Applying b
Applying a
Applying b
Applying a
Applying b
Applying a


{0: ('P', (True, [1, 9, 4])),
 1: ('bP', (True, [0, 2, 18])),
 2: ('bbP', (True, [1, 3, 37])),
 3: ('bbbP', (True, [2, 4, 75])),
 4: ('abP', (True, [0, 7, 3])),
 7: ('abbP', (True, [8, 150, 4])),
 8: ('abbbP', (True, [9, 226, 7])),
 9: ('aabP', (True, [8, 0, 17])),
 17: ('aabbP', (True, [9, 18, 301])),
 18: ('aabbbP', (True, [1, 36, 17])),
 36: ('baabbP', (True, [18, 37, 527])),
 37: ('baabbbP', (True, [2, 74, 36])),
 74: ('bbaabbP', (True, [75, 1055, 37])),
 75: ('bbaabbbP', (True, [74, 3, 150])),
 150: ('bbbaabbP', (True, [225, 75, 7])),
 225: ('abbaabbP', (True, [226, 1055, 150])),
 226: ('abbaabbbP', (True, [8, 301, 225])),
 301: ('abbbaabbP', (True, [17, 527, 226])),
 527: ('aabbaabbP', (True, [301, 36, 1055])),
 1055: ('baabbaabbP', (True, [225, 527, 74]))}